In [1]:
import pandas as pd
import re
import altair as alt
pd.set_option("display.max_rows", 300)

In [2]:
# downloaded from https://frontex.europa.eu/we-know/migratory-map/ on 2022-02-08 -- max month DEC 2021
df=pd.read_csv('../data/Frontex-Monthly_detections_of_IBC_20220204.csv', delimiter=';')
df.head()

,Route,Border type or inland,Nationality,JAN2009,FEB2009,MAR2009,APR2009,MAY2009,JUN2009,JUL2009,...,MAR2021,APR2021,MAY2021,JUN2021,JUL2021,AUG2021,SEP2021,OCT2021,NOV2021,DEC2021
0,Western Balkan Route,Land,Syria,0,0,0,0,0,0,0,...,1657,2424,2505,2440,2490,3651,6113,5913,4317,3018
1,Western Balkan Route,Land,Afghanistan,0,0,54,34,51,35,75,...,428,415,354,793,904,1809,1951,1247,1487,2013
2,Western Balkan Route,Land,Turkey,3,4,3,0,4,5,9,...,30,22,44,44,109,93,226,311,381,370
3,Western Balkan Route,Land,India,0,0,0,0,0,0,0,...,5,0,2,20,14,26,23,107,102,245
4,Western Balkan Route,Land,Tunisia,1,0,0,0,0,0,0,...,22,18,11,13,20,27,54,189,203,237


## Data preparation

In [3]:
# make monthly columns into row
df.columns

Index(['Route', 'Border type or inland', 'Nationality', 'JAN2009', 'FEB2009',
       'MAR2009', 'APR2009', 'MAY2009', 'JUN2009', 'JUL2009',
       ...
       'MAR2021', 'APR2021', 'MAY2021', 'JUN2021', 'JUL2021', 'AUG2021',
       'SEP2021', 'OCT2021', 'NOV2021', 'DEC2021'],
      dtype='object', length=159)

In [4]:
dfm = df.melt(id_vars=['Route', 'Border type or inland', 'Nationality'])
dfm.head()

,Route,Border type or inland,Nationality,variable,value
0,Western Balkan Route,Land,Syria,JAN2009,0
1,Western Balkan Route,Land,Afghanistan,JAN2009,0
2,Western Balkan Route,Land,Turkey,JAN2009,3
3,Western Balkan Route,Land,India,JAN2009,0
4,Western Balkan Route,Land,Tunisia,JAN2009,1


In [5]:
dfm.columns=(['Route', 'BorderType','Nationality','month-year','DetectedCrossings'])

In [6]:
dfm.head()

,Route,BorderType,Nationality,month-year,DetectedCrossings
0,Western Balkan Route,Land,Syria,JAN2009,0
1,Western Balkan Route,Land,Afghanistan,JAN2009,0
2,Western Balkan Route,Land,Turkey,JAN2009,3
3,Western Balkan Route,Land,India,JAN2009,0
4,Western Balkan Route,Land,Tunisia,JAN2009,1


In [7]:
# replace MMM code by number
dfm['month-year-reconfig']=[re.sub(r'(JAN)(\d\d\d\d)', '\g<2>-01', date) for date in dfm['month-year']]
dfm['month-year-reconfig']=[re.sub(r'(FEB)(\d\d\d\d)', '\g<2>-02', date) for date in dfm['month-year-reconfig']]
dfm['month-year-reconfig']=[re.sub(r'(MAR)(\d\d\d\d)', '\g<2>-03', date) for date in dfm['month-year-reconfig']]
dfm['month-year-reconfig']=[re.sub(r'(APR)(\d\d\d\d)', '\g<2>-04', date) for date in dfm['month-year-reconfig']]
dfm['month-year-reconfig']=[re.sub(r'(MAY)(\d\d\d\d)', '\g<2>-05', date) for date in dfm['month-year-reconfig']]
dfm['month-year-reconfig']=[re.sub(r'(JUN)(\d\d\d\d)', '\g<2>-06', date) for date in dfm['month-year-reconfig']]
dfm['month-year-reconfig']=[re.sub(r'(JUL)(\d\d\d\d)', '\g<2>-07', date) for date in dfm['month-year-reconfig']]
dfm['month-year-reconfig']=[re.sub(r'(AUG)(\d\d\d\d)', '\g<2>-08', date) for date in dfm['month-year-reconfig']]
dfm['month-year-reconfig']=[re.sub(r'(SEP)(\d\d\d\d)', '\g<2>-09', date) for date in dfm['month-year-reconfig']]
dfm['month-year-reconfig']=[re.sub(r'(OCT)(\d\d\d\d)', '\g<2>-10', date) for date in dfm['month-year-reconfig']]
dfm['month-year-reconfig']=[re.sub(r'(NOV)(\d\d\d\d)', '\g<2>-11', date) for date in dfm['month-year-reconfig']]
dfm['month-year-reconfig']=[re.sub(r'(DEC)(\d\d\d\d)', '\g<2>-12', date) for date in dfm['month-year-reconfig']]

In [8]:
# convert to date
dfm['month-year-reconfig-parsed'] = pd.to_datetime(dfm['month-year-reconfig'], format='%Y-%m')

In [9]:
dfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129948 entries, 0 to 129947
Data columns (total 7 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Route                       129948 non-null  object        
 1   BorderType                  129948 non-null  object        
 2   Nationality                 129948 non-null  object        
 3   month-year                  129948 non-null  object        
 4   DetectedCrossings           129948 non-null  int64         
 5   month-year-reconfig         129948 non-null  object        
 6   month-year-reconfig-parsed  129948 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 6.9+ MB


In [10]:
# merge with table holding country - region association
regions = pd.read_csv('../data/Frontex-adjusted_country-and-continent-codes-list.csv', delimiter=';')
regions.head()

,continent,region,subregion,country,M49 code,ISO-alpha3 code,ISO-alpha2 code,Other groupings
0,Asia,Southern Asia,Southern Asia,Afghanistan,4,AFG,AF,LDC LLDC
1,Europe,Southern Europe,Southern Europe,Albania,8,ALB,AL,NaN
2,Americas,Northern America,Northern America,Antarctica,10,ATA,AQ,NaN
3,Africa,Northern Africa,Northern Africa,Algeria,12,DZA,DZ,NaN
4,Oceania,Polynesia,Polynesia,American Samoa,16,ASM,AS,SIDS


In [11]:
dfmm = dfm.merge(regions, left_on='Nationality',right_on='country')

In [12]:
len(df.Nationality.value_counts())

152

In [13]:
#three missing Nationalities: Unknown, Stateless, Third-country
len(dfmm.Nationality.value_counts())

149

In [14]:
dfmm.head()

,Route,BorderType,Nationality,month-year,DetectedCrossings,month-year-reconfig,month-year-reconfig-parsed,continent,region,subregion,country,M49 code,ISO-alpha3 code,ISO-alpha2 code,Other groupings
0,Western Balkan Route,Land,Syria,JAN2009,0,2009-01,2009-01-01,Asia,Western Asia,Western Asia,Syria,760,SYR,SY,NaN
1,Central Mediterranean Route,Sea,Syria,JAN2009,0,2009-01,2009-01-01,Asia,Western Asia,Western Asia,Syria,760,SYR,SY,NaN
2,Western African Route,Sea,Syria,JAN2009,0,2009-01,2009-01-01,Asia,Western Asia,Western Asia,Syria,760,SYR,SY,NaN
3,Eastern Mediterranean Route,Land,Syria,JAN2009,82,2009-01,2009-01-01,Asia,Western Asia,Western Asia,Syria,760,SYR,SY,NaN
4,Eastern Mediterranean Route,Sea,Syria,JAN2009,9,2009-01,2009-01-01,Asia,Western Asia,Western Asia,Syria,760,SYR,SY,NaN


In [15]:
dfmm.columns

Index(['Route', 'BorderType', 'Nationality', 'month-year', 'DetectedCrossings',
       'month-year-reconfig', 'month-year-reconfig-parsed', 'continent',
       'region', 'subregion', 'country', 'M49 code', 'ISO-alpha3 code',
       'ISO-alpha2 code', 'Other groupings'],
      dtype='object')

In [16]:
dfmm=dfmm[['Route', 'BorderType', 'Nationality',  'month-year-reconfig-parsed','DetectedCrossings','continent','region',  'M49 code', 'ISO-alpha3 code','ISO-alpha2 code']]

In [17]:
dfmm.columns

Index(['Route', 'BorderType', 'Nationality', 'month-year-reconfig-parsed',
       'DetectedCrossings', 'continent', 'region', 'M49 code',
       'ISO-alpha3 code', 'ISO-alpha2 code'],
      dtype='object')

In [18]:
dfmm.columns=(['Route', 'BorderType', 'Nationality', 'month-year-reconfig-parsed',
       'DetectedCrossings', 'continent', 'region', 'numericISO',
       'ISO3', 'ISO2'])

## Crossings by region in 2020

In [73]:
#filter for 2020
dfmm_2020 = dfmm[(dfmm['month-year-reconfig-parsed']>='2015-01-01')&(dfmm['month-year-reconfig-parsed']<='2015-12-01')]

In [74]:
dfmm_2020['month-year-reconfig-parsed'].min()

Timestamp('2015-01-01 00:00:00')

In [75]:
dfmm_2020['month-year-reconfig-parsed'].max()

Timestamp('2015-12-01 00:00:00')

In [67]:
dfmm_2020_by_region = dfmm_2020.groupby(by='region')['DetectedCrossings'].sum().reset_index()
dfmm_2020_by_region

,region,DetectedCrossings
0,Australia and New Zealand,0
1,Central America,0
2,Central Asia,128
3,Eastern Asia,13
4,Eastern Europe,484
5,Latin America and the Caribbean,105
6,Melanesia,3
7,Micronesia,0
8,Northern Africa,59792
9,Northern America,3


In [76]:
dfm_unknown = dfm[(dfm['Nationality']=='Unknown')|(dfm['Nationality']=='Stateless')| (dfm['Nationality']=='Third-country')]
dfmm_2020_unknown = dfm_unknown[(dfm_unknown['month-year-reconfig-parsed']>='2015-01-01')&(dfm_unknown['month-year-reconfig-parsed']<='2015-12-01')]

In [77]:
dfmm_2020_unknown['DetectedCrossings'].sum()

556434

In [78]:
overall_detected_crossings_2020 = dfmm_2020_unknown['DetectedCrossings'].sum() + dfmm_2020_by_region['DetectedCrossings'].sum()
overall_detected_crossings_2020

756202

In [79]:
dfmm_2020_by_region['share_in_%'] = dfmm_2020_by_region['DetectedCrossings']/overall_detected_crossings_2020*100

In [80]:
dfmm_2020_by_region

,region,DetectedCrossings,share_in_%
0,Australia and New Zealand,0,0.000000
1,Central America,0,0.000000
2,Central Asia,128,0.016927
3,Eastern Asia,13,0.001719
4,Eastern Europe,484,0.064004
5,Latin America and the Caribbean,105,0.013885
6,Melanesia,3,0.000397
7,Micronesia,0,0.000000
8,Northern Africa,59792,7.906882
9,Northern America,3,0.000397


## Crossings from African nationals

In [19]:
# filter for African countries
dfmm_africa = dfmm[dfmm['continent']=='Africa']
dfmm_africa.head()


,Route,BorderType,Nationality,month-year-reconfig-parsed,DetectedCrossings,continent,region,numericISO,ISO3,ISO2
6084,Western Balkan Route,Land,Tunisia,2009-01-01,1,Africa,Northern Africa,788,TUN,TN
6085,Central Mediterranean Route,Sea,Tunisia,2009-01-01,686,Africa,Northern Africa,788,TUN,TN
6086,Western African Route,Sea,Tunisia,2009-01-01,0,Africa,Northern Africa,788,TUN,TN
6087,Eastern Mediterranean Route,Land,Tunisia,2009-01-01,1,Africa,Northern Africa,788,TUN,TN
6088,Eastern Mediterranean Route,Sea,Tunisia,2009-01-01,0,Africa,Northern Africa,788,TUN,TN


In [20]:
# per route, create key
dfmm_africa['route-bordertype'] = dfmm_africa['Route']+'-'+dfmm_africa['BorderType']
dfmm_africa.head()

/var/folders/fk/_qbrt9ys18b6s2yb_kqblv1xt9_xg3/T/ipykernel_81633/409651077.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmm_africa['route-bordertype'] = dfmm_africa['Route']+'-'+dfmm_africa['BorderType']


,Route,BorderType,Nationality,month-year-reconfig-parsed,DetectedCrossings,continent,region,numericISO,ISO3,ISO2,route-bordertype
6084,Western Balkan Route,Land,Tunisia,2009-01-01,1,Africa,Northern Africa,788,TUN,TN,Western Balkan Route-Land
6085,Central Mediterranean Route,Sea,Tunisia,2009-01-01,686,Africa,Northern Africa,788,TUN,TN,Central Mediterranean Route-Sea
6086,Western African Route,Sea,Tunisia,2009-01-01,0,Africa,Northern Africa,788,TUN,TN,Western African Route-Sea
6087,Eastern Mediterranean Route,Land,Tunisia,2009-01-01,1,Africa,Northern Africa,788,TUN,TN,Eastern Mediterranean Route-Land
6088,Eastern Mediterranean Route,Sea,Tunisia,2009-01-01,0,Africa,Northern Africa,788,TUN,TN,Eastern Mediterranean Route-Sea


# Data analysis

## How has the number of detected crossings from African nationals evolved over time?

In [21]:
crossings_african_nationals = dfmm_africa.groupby(by='month-year-reconfig-parsed')['DetectedCrossings'].sum().reset_index()
crossings_african_nationals

,month-year-reconfig-parsed,DetectedCrossings
0,2009-01-01,2655
1,2009-02-01,2536
2,2009-03-01,4135
3,2009-04-01,2883
4,2009-05-01,1648
5,2009-06-01,1617
6,2009-07-01,1932
7,2009-08-01,1879
8,2009-09-01,1935
9,2009-10-01,2702


In [22]:
alt.Chart(crossings_african_nationals).mark_line().encode(
    x='month-year-reconfig-parsed:T',
    y='DetectedCrossings'
)

alt.Chart(...)

In [23]:
crossings_african_nationals['year']=crossings_african_nationals['month-year-reconfig-parsed'].dt.year

In [24]:
crossings_african_nationals['year_parsed']=pd.to_datetime(crossings_african_nationals['year'], format='%Y')

In [25]:
crossings_african_nationals_yearly=crossings_african_nationals.groupby(by='year_parsed')['DetectedCrossings'].sum().reset_index()

In [26]:
crossings_african_nationals_yearly

,year_parsed,DetectedCrossings
0,2009-01-01,26092
1,2010-01-01,22644
2,2011-01-01,80373
3,2012-01-01,25834
4,2013-01-01,41985
5,2014-01-01,128922
6,2015-01-01,172191
7,2016-01-01,184390
8,2017-01-01,129697
9,2018-01-01,81866


In [27]:
alt.Chart(crossings_african_nationals_yearly, title='Frontex Detected Crossings from African nationals').mark_bar(width=20).encode(
    x='year_parsed',
    y='DetectedCrossings',
)

alt.Chart(...)

In [28]:
# filter for countries that receive EUTF money

In [29]:
edf = pd.read_csv('../data/EUTF-country-budget.csv',delimiter=',')
edf.head()

,country,money_from_all_documented_projects(reg+single),ISO
0,Somalia,590870442.2,SOM
1,Sudan,553485934.9,SDN
2,Libya,396428927.3,LBY
3,Ethiopia,334563566.0,ETH
4,Niger,294621000.0,NER


In [30]:
dfmm_africa.head()

,Route,BorderType,Nationality,month-year-reconfig-parsed,DetectedCrossings,continent,region,numericISO,ISO3,ISO2,route-bordertype
6084,Western Balkan Route,Land,Tunisia,2009-01-01,1,Africa,Northern Africa,788,TUN,TN,Western Balkan Route-Land
6085,Central Mediterranean Route,Sea,Tunisia,2009-01-01,686,Africa,Northern Africa,788,TUN,TN,Central Mediterranean Route-Sea
6086,Western African Route,Sea,Tunisia,2009-01-01,0,Africa,Northern Africa,788,TUN,TN,Western African Route-Sea
6087,Eastern Mediterranean Route,Land,Tunisia,2009-01-01,1,Africa,Northern Africa,788,TUN,TN,Eastern Mediterranean Route-Land
6088,Eastern Mediterranean Route,Sea,Tunisia,2009-01-01,0,Africa,Northern Africa,788,TUN,TN,Eastern Mediterranean Route-Sea


In [31]:
len(edf)

35

In [32]:
dfmm_africa_edf = dfmm_africa.merge(edf, left_on='ISO3', right_on='ISO', how='outer')
dfmm_africa_edf.head()

,Route,BorderType,Nationality,month-year-reconfig-parsed,DetectedCrossings,continent,region,numericISO,ISO3,ISO2,route-bordertype,country,money_from_all_documented_projects(reg+single),ISO
0,Western Balkan Route,Land,Tunisia,2009-01-01,1,Africa,Northern Africa,788,TUN,TN,Western Balkan Route-Land,Tunisia,76450000.0,TUN
1,Central Mediterranean Route,Sea,Tunisia,2009-01-01,686,Africa,Northern Africa,788,TUN,TN,Central Mediterranean Route-Sea,Tunisia,76450000.0,TUN
2,Western African Route,Sea,Tunisia,2009-01-01,0,Africa,Northern Africa,788,TUN,TN,Western African Route-Sea,Tunisia,76450000.0,TUN
3,Eastern Mediterranean Route,Land,Tunisia,2009-01-01,1,Africa,Northern Africa,788,TUN,TN,Eastern Mediterranean Route-Land,Tunisia,76450000.0,TUN
4,Eastern Mediterranean Route,Sea,Tunisia,2009-01-01,0,Africa,Northern Africa,788,TUN,TN,Eastern Mediterranean Route-Sea,Tunisia,76450000.0,TUN


In [33]:
euft_african_countries = dfmm_africa_edf[dfmm_africa_edf['country'].notnull()]
euft_african_countries.head()

,Route,BorderType,Nationality,month-year-reconfig-parsed,DetectedCrossings,continent,region,numericISO,ISO3,ISO2,route-bordertype,country,money_from_all_documented_projects(reg+single),ISO
0,Western Balkan Route,Land,Tunisia,2009-01-01,1,Africa,Northern Africa,788,TUN,TN,Western Balkan Route-Land,Tunisia,76450000.0,TUN
1,Central Mediterranean Route,Sea,Tunisia,2009-01-01,686,Africa,Northern Africa,788,TUN,TN,Central Mediterranean Route-Sea,Tunisia,76450000.0,TUN
2,Western African Route,Sea,Tunisia,2009-01-01,0,Africa,Northern Africa,788,TUN,TN,Western African Route-Sea,Tunisia,76450000.0,TUN
3,Eastern Mediterranean Route,Land,Tunisia,2009-01-01,1,Africa,Northern Africa,788,TUN,TN,Eastern Mediterranean Route-Land,Tunisia,76450000.0,TUN
4,Eastern Mediterranean Route,Sea,Tunisia,2009-01-01,0,Africa,Northern Africa,788,TUN,TN,Eastern Mediterranean Route-Sea,Tunisia,76450000.0,TUN


In [34]:
len(euft_african_countries.Nationality.value_counts())

35

In [35]:
crossings_eutf_recipients = euft_african_countries.groupby(by='month-year-reconfig-parsed')['DetectedCrossings'].sum().reset_index()

In [36]:
crossings_eutf_recipients

,month-year-reconfig-parsed,DetectedCrossings
0,2009-01-01,2592
1,2009-02-01,2521
2,2009-03-01,4130
3,2009-04-01,2876
4,2009-05-01,1634
5,2009-06-01,1609
6,2009-07-01,1922
7,2009-08-01,1841
8,2009-09-01,1913
9,2009-10-01,2697


In [37]:
crossings_eutf_recipients['year']=crossings_eutf_recipients['month-year-reconfig-parsed'].dt.year
crossings_eutf_recipients['year_parsed']=pd.to_datetime(crossings_eutf_recipients['year'], format='%Y')
crossings_eutf_recipients_yearly=crossings_eutf_recipients.groupby(by='year_parsed')['DetectedCrossings'].sum().reset_index()

In [38]:
alt.Chart(crossings_eutf_recipients_yearly, title='Frontex Detected Crossings from African nationals from EUTF recipient countries').mark_bar(width=20, color='orange').encode(
    x='year_parsed',
    y='DetectedCrossings',
)

alt.Chart(...)

In [39]:
africa = alt.Chart(crossings_african_nationals_yearly, title='Frontex Detected Crossings from all African nationals (blue) and EUTF recipient countries (orange) ').mark_bar(width=20).encode(
    x='year_parsed',
    y='DetectedCrossings',
)

eutf = alt.Chart(crossings_eutf_recipients_yearly).mark_bar(width=7, color='orange').encode(
    x='year_parsed',
    y='DetectedCrossings',
)

combination = africa + eutf
combination

alt.LayerChart(...)

In [40]:
crossings_african_nationals_yearly

,year_parsed,DetectedCrossings
0,2009-01-01,26092
1,2010-01-01,22644
2,2011-01-01,80373
3,2012-01-01,25834
4,2013-01-01,41985
5,2014-01-01,128922
6,2015-01-01,172191
7,2016-01-01,184390
8,2017-01-01,129697
9,2018-01-01,81866


In [41]:
crossings_eutf_recipients_yearly

,year_parsed,DetectedCrossings
0,2009-01-01,25900
1,2010-01-01,22396
2,2011-01-01,68382
3,2012-01-01,24710
4,2013-01-01,41384
5,2014-01-01,101311
6,2015-01-01,160823
7,2016-01-01,181546
8,2017-01-01,127248
9,2018-01-01,80088
